<a href="https://colab.research.google.com/github/Tomato-in-Japan/Subject_Fri1/blob/Homeworks/%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%82%B9%E5%85%A5%E9%96%80_%E7%AC%AC09%E5%9B%9E_%E3%83%AF%E3%83%BC%E3%82%AF%E3%82%B7%E3%83%BC%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第9回 データサイエンス入門 ワークシート

次のセルに「学籍番号」、「氏名」、「クラス」、「出席番号」を入力して下さい。

In [ ]:
# 学籍番号：b2220630　氏名：加賀大智　クラス：2A　出席番号：17

# 1. 導入

## 1-1. 概要

第7回～第8回では線形重回帰分析の基本について学びました。今回（第9回）は線形重回帰による実データの分析を行います。

実データの分析では「異常値の削除」、「データ型の変換」などの”前処理”が必要です。
本ワークシートの事前解説動画（講義ビデオ「９回目」）を用意していますので、確認してください。

必要に応じて「ウェブなどで調べる」、「修学支援室／メディアコンサルタントで質問する」などしてください。

## 1-2. 本ワークシートの使い方

基本的に、ワークシート単体で学習が可能な設計になっています。

1.   入力済みセル
2.   穴埋め用セル
3.   自由記述用セル

がありますので、手を動かしながら学習を進めて下さい。

## 1-3. ライブラリのインポート

本ワークシートで使うライブラリをインポートします。以下のセルを実行して下さい。

In [1]:
# データの加工・処理・分析ライブラリ
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series, DataFrame
import pandas as pd

# 可視化ライブラリ
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# 機械学習ライブラリ
import sklearn

# 小数点第 3 位まで表示
%precision 3

'%.3f'

# 2. 知識定着

## 2-1. 準備

以下の URL で公開されている自動車の価格と付随する属性（自動車の重さなど）の分析を行います：

https://networkrepository.com/autos-imports-85.php

ポータルで再配布しているファイル「autos_data.csv」を利用してください。

第7回～第8回を参考に、csv ファイルを pandas の DataFrame として読み込んでください。

In [2]:
# コード1（模範解答は最下部）
# csv ファイルを pandas の DataFrame オブジェクト df として読み込み
df = pd.read_csv('autos_data.csv')

まずは df の中身を確認してみましょう。

In [3]:
# コード2（模範解答は最下部）
# df を表示
print(df)

     symboling normalized_losses         make fuel_type aspiration  \
0            3                 ?  alfa-romero       gas        std   
1            3                 ?  alfa-romero       gas        std   
2            1                 ?  alfa-romero       gas        std   
3            2               164         audi       gas        std   
4            2               164         audi       gas        std   
..         ...               ...          ...       ...        ...   
200         -1                95        volvo       gas        std   
201         -1                95        volvo       gas      turbo   
202         -1                95        volvo       gas        std   
203         -1                95        volvo    diesel      turbo   
204         -1                95        volvo       gas      turbo   

    num_of_doors   body_style drive_wheels engine_location  wheel_base  ...  \
0            two  convertible          rwd           front        88.6  ...   
1

「205 rows × 26 columns」とあるので、205行、26列のデータであることが分かります。

## 2-2. 問題設定

【例題】目的変数を「price」、説明変数を「horsepower、width、height」として、線形重回帰分析を実行せよ。

## 2-3. 前処理

我々の目的は「horsepower、width、height」から「price」を予測する事なので、4 つのカラム「horsepower」、「width」、「height」、「price」以外は不要です。df から4 つのカラム「horsepower」、「width」、「height」、「price」以外を削除して下さい。

In [7]:
# コード3（模範解答は最下部）
# 4 つのカラム「horsepower」、「width」、「height」、「price」以外を削除
df=df[["horsepower","width","height","price"]]
# df を表示（確認）
print(df)

    horsepower  width  height  price
0          111   64.1    48.8  13495
1          111   64.1    48.8  16500
2          154   65.5    52.4  16500
3          102   66.2    54.3  13950
4          115   66.4    54.3  17450
..         ...    ...     ...    ...
200        114   68.9    55.5  16845
201        160   68.8    55.5  19045
202        134   68.9    55.5  21485
203        106   68.9    55.5  22470
204        114   68.9    55.5  22625

[205 rows x 4 columns]


## 2-3-1. 異常値の削除

「2-1. 準備」で「df を表示」しました。このとき、データの中に「?」という成分がある事が分かります。これは”欠損値”あるいは”異常値”です。
実際のデータ分析でも、生のデータを眺める事は重要です。

各カラムに「?」が何個あるかカウントします。以下のセルを実行してください。

In [8]:
# 各カラムに「?」が何個あるかカウント
df.isin(['?']).sum()

horsepower    2
width         0
height        0
price         4
dtype: int64

「?」がある行を削除します。以下のセルを実行して下さい。

In [9]:
# 「?」がある行を削除
df = df.replace('?', np.nan).dropna()

# 各カラムに「?」が何個あるかカウント
df.isin(['?']).sum()

horsepower    0
width         0
height        0
price         0
dtype: int64

## 2-3-2 データ型の変換

以下のセルを実行して下さい。

In [10]:
# df のデータ型の確認
df.dtypes

horsepower     object
width         float64
height        float64
price          object
dtype: object

2つのカラム「price」、「horsepower」が数値型ではないので、to_numeric を使って数値型に変換します。以下のセルを実行して下さい。

In [11]:
df = df.assign(price=pd.to_numeric(df.price))
df = df.assign(horsepower=pd.to_numeric(df.horsepower))

実際のデータの分析においても、このようにデータの型が整っていない事があるので注意が必要です。

変更後の df のデータ型を確認して下さい。

In [13]:
# コード4（模範解答は最下部）
# df のデータ型の確認
df.dtypes

horsepower      int64
width         float64
height        float64
price           int64
dtype: object

## 2-3-3 相関係数の確認

以下のセルを実行して下さい。

In [15]:
# コード5（模範解答は最下部）
# 各カラム（列）同士の相関係数の計算
df.corr()

,horsepower,width,height,price
horsepower,1.000000,0.615315,-0.087407,0.810533
width,0.615315,1.000000,0.309223,0.753871
height,-0.087407,0.309223,1.000000,0.134990
price,0.810533,0.753871,0.134990,1.000000


ここで注目するべきなのは、説明変数「horsepower」、「width」、「height」同士の相関係数です。

なぜならば、多重共線性が生じる可能性を考慮しなければならないからです。

多重共線性とは、重回帰において、説明変数の中に相関係数が高い組み合わせが存在することいいます。このとき、偏回帰係数の分散が大きくなり、係数の有意性が失われます。

ここでは、「width」と「horsepower」の相関係数が 0.615315 とやや高めですが、このままモデルを構築します。

## 2-4 モデルの構築と評価

データの前処理が終わったのでモデルの構築を行います。ここからが機械学習（重回帰）の本番です。

機械学習ライブラリ「scikit-learn」を使用する事により、「重回帰」を簡単に実装する事が出来ます。

今回は「訓練データ：テストデータ＝５：５」とし、訓練データを標準化します。

In [16]:
# 必要なライブラリのインポート
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# 目的変数に「price」、説明変数に「horsepower」、「width」、「height」を指定
x = df.drop('price', axis=1)
y = df['price']

# データを訓練データとテストデータに分ける
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=0)

x_train と x_test を標準化し、x_train を表示してください。

In [21]:
# コード6（模範解答は最下部）
# データを標準化するライブラリのインポート
from sklearn import preprocessing
ss=preprocessing.StandardScaler()

# x_train を標準化し x_train に格納
ss.fit(x_train)
x_train=ss.transform(x_train)

# x_test を標準化し x_test に格納
ss.fit(x_test)
x_test=ss.transform(x_test)

# x_train を表示
print(x_train)

[[ 0.362  0.646  1.027]
 [-0.985 -0.721  0.194]
 [ 0.274 -0.766 -1.908]
 [-0.956 -0.903 -0.044]
 [-0.195  1.193  1.225]
 [-0.956 -0.903 -0.123]
 [ 0.625  2.059  1.146]
 [-0.224 -0.174 -0.242]
 [-0.048  0.099 -1.036]
 [ 1.562  2.15  -1.115]
 [-0.985 -0.721  0.194]
 [-0.985 -0.721  0.194]
 [ 0.42  -0.174 -0.798]
 [ 0.567  0.51   0.828]
 [-0.927 -0.63  -0.242]
 [-0.985 -0.903 -1.194]
 [ 0.245  0.51   0.59 ]
 [-0.839 -0.903  0.828]
 [-0.985 -0.903 -1.115]
 [ 2.352  0.965  0.035]
 [-0.4   -0.174 -1.67 ]
 [-0.136  1.193  1.225]
 [-0.019 -0.037 -1.591]
 [ 1.474  0.327  0.59 ]
 [ 0.01  -0.903 -1.115]
 [ 0.01   0.191  0.273]
 [ 0.01  -0.903 -1.115]
 [ 0.42  -0.083 -0.639]
 [ 0.127  1.421  0.749]
 [ 0.245  2.56   0.828]
 [ 1.562  2.697  1.146]
 [ 0.42  -0.174 -0.798]
 [ 1.269  0.236 -1.353]
 [-0.927 -0.63  -0.242]
 [-0.282  0.327  0.511]
 [-0.927 -0.63  -0.322]
 [-0.956 -0.903  0.353]
 [-0.282  0.327  0.115]
 [ 0.567 -0.447  0.273]
 [-0.985 -0.903 -1.115]
 [ 2.879  0.965 -1.551]
 [-0.751 -0.858 

第08回を参考に、モデルの初期化と学習を実施してください。

In [25]:
# コード7（模範解答は最下部）
# モデルの初期化と学習
model=LinearRegression()
model.fit(x_train,y_train)

LinearRegression()

以上で学習は完了です。

学習結果の評価を行います。score メソッドを利用する事により決定係数を計算できます。

決定係数はデータに対する、推定されたモデルの当てはまりの良さ（度合い）を表し、$0$ から $1$ までの値をとります。$1$ に近いほど、当てはまりが良いです。

ここで、「決定係数はいくつ以上なら良いのか？」という疑問が生じますが、この問いに対する絶対的な回答は存在しません。

ここでは、参考として「$R^2 \geq 0.8$：精度良い」、「$R^2 \geq 0.5$：精度やや良い」、「$R^2 < 0.5$：精度良くない」という基準を紹介します（cf. https://istat.co.jp/ta_commentary/multiple_02 ）。

では、実際に決定係数を計算してみましょう。以下のセルを実行して下さい。

In [26]:
print('訓練データに対する決定係数：', model.score(x_train, y_train))
print('テストデータに対する決定係数：', model.score(x_test, y_test))

訓練データに対する決定係数： 0.7333575683901377
テストデータに対する決定係数： 0.6836797149339668


「訓練データに対する決定係数」と「テストデータに対する決定係数」が近い値を取っている事から、過学習が起きていないと判断出来ます。

過学習とは、機械学習で、訓練データに対してあまりに適合し過ぎて本来データが示唆する傾向から大きく外れてしまう現象です。つまり、未知のデータに対する予測が適切に行えない状態です。

標準化した説明変数から算出される偏回帰係数を**標準偏回帰係数**と呼びます。標準偏回帰係数は各変数の重要度を表す指標であり、大小の比較に意味があります。

また、以下のコードを実行すると、偏回帰係数を確認することが出来ます。

In [27]:
# 偏回帰係数
pd.Series(model.coef_, index=x.columns)

horsepower    2788.946965
width         4013.955388
height         578.718631
dtype: float64

In [28]:
# 偏回帰係数（定数項、切片）
model.intercept_

12521.212121212122

# 3. 知識活用（課題）

【問題】目的変数を「price」、説明変数を「width、engine_size」として、線形重回帰分析を実行し、決定係数を求めよ。ただし、$(訓練データ):(テストデータ) = 7:3$ および random_state=0 とし、説明変数は標準化せよ。

In [103]:
# セルは必要に応じて追加して下さい。
dfa = pd.read_csv('autos_data.csv')
df=dfa[["width","engine_size","price"]]
df.isin(['?']).sum()
df = df.replace('?', np.nan).dropna()
df.isin(['?']).sum()
df.dtypes

width          float64
engine_size      int64
price           object
dtype: object

In [104]:
df = df.assign(price=pd.to_numeric(df.price))
df.dtypes

width          float64
engine_size      int64
price            int64
dtype: object

In [105]:
df.corr()

,width,engine_size,price
width,1.000000,0.729436,0.751265
engine_size,0.729436,1.000000,0.872335
price,0.751265,0.872335,1.000000


In [106]:
x = df.drop('price', axis=1)
y = df['price']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
ss.fit(x_train)
x_train=ss.transform(x_train)
ss.fit(x_test)
x_test=ss.transform(x_test)
print(x_train)

[[-0.3   -0.135]
 [-0.155 -0.412]
 [-1.074 -0.891]
 [-0.687 -0.689]
 [-0.977 -0.714]
 [-0.059 -1.394]
 [-0.107  0.519]
 [-0.784 -0.866]
 [-0.977 -0.714]
 [-0.204 -0.438]
 [-0.977 -0.714]
 [-0.204 -0.438]
 [ 1.005  1.4  ]
 [-0.204 -0.438]
 [-0.977 -0.714]
 [ 1.247  0.67 ]
 [ 0.522  2.105]
 [-0.591 -0.085]
 [-0.204 -0.438]
 [ 0.522  0.267]
 [ 2.311  5.051]
 [-0.204 -0.438]
 [ 1.489  0.393]
 [ 1.247 -0.135]
 [ 0.135  0.368]
 [-0.155  0.67 ]
 [ 0.328 -0.085]
 [-0.977 -0.438]
 [ 1.489  1.199]
 [-0.3   -0.161]
 [-0.494 -0.438]
 [ 0.667  0.116]
 [-0.687 -0.84 ]
 [ 1.44   0.393]
 [-0.155 -0.714]
 [ 0.908  1.149]
 [ 0.328  0.167]
 [-0.977 -0.714]
 [ 1.247 -0.135]
 [-0.687 -0.689]
 [ 0.667  0.393]
 [-0.784 -0.866]
 [-0.832  0.116]
 [-0.977 -0.714]
 [ 1.247 -0.135]
 [-0.977 -0.714]
 [ 2.166  1.451]
 [-0.107  0.519]
 [ 0.086 -0.387]
 [ 2.262  2.735]
 [-0.784 -0.866]
 [-0.784 -0.866]
 [-0.204 -0.387]
 [ 0.522  0.972]
 [-0.881 -0.689]
 [-0.977 -0.891]
 [-0.881 -0.412]
 [-1.074 -0.438]
 [-0.977 -0.89

In [107]:
model=LinearRegression()
model.fit(x_train,y_train)
print('訓練データに対する決定係数：', model.score(x_train, y_train))
print('テストデータに対する決定係数：', model.score(x_test, y_test))

訓練データに対する決定係数： 0.7834508035929952
テストデータに対する決定係数： 0.6972739633127203


In [ ]:
pd.Series(model.coef_, index=x.columns)

# 4. 知識応用（発展課題）

【問題】目的変数を「price」、説明変数を「各自で設定」として、決定係数が大きくなるように重回帰分析を実行し、決定係数を求めよ。また、その結果を考察せよ。

In [108]:
# セルは必要に応じて追加して下さい。
df = pd.read_csv('autos_data.csv')
df = df[["num_of_doors","length","horsepower","price"]]
print(df)
df = df.replace('two',2)
df = df.replace('four',4)
df = df.replace('?', np.nan).dropna()
print(df)
df.dtypes

    num_of_doors  length horsepower  price
0            two   168.8        111  13495
1            two   168.8        111  16500
2            two   171.2        154  16500
3           four   176.6        102  13950
4           four   176.6        115  17450
..           ...     ...        ...    ...
200         four   188.8        114  16845
201         four   188.8        160  19045
202         four   188.8        134  21485
203         four   188.8        106  22470
204         four   188.8        114  22625

[205 rows x 4 columns]
     num_of_doors  length horsepower  price
0             2.0   168.8        111  13495
1             2.0   168.8        111  16500
2             2.0   171.2        154  16500
3             4.0   176.6        102  13950
4             4.0   176.6        115  17450
..            ...     ...        ...    ...
200           4.0   188.8        114  16845
201           4.0   188.8        160  19045
202           4.0   188.8        134  21485
203           4.0   

num_of_doors    float64
length          float64
horsepower       object
price            object
dtype: object

In [109]:
df = df.assign(price=pd.to_numeric(df.price))
df = df.assign(horsepower=pd.to_numeric(df.horsepower))
df.dtypes

num_of_doors    float64
length          float64
horsepower        int64
price             int64
dtype: object

In [110]:
df.corr()

,num_of_doors,length,horsepower,price
num_of_doors,1.000000,0.404454,-0.099274,0.046571
length,0.404454,1.000000,0.586110,0.694556
horsepower,-0.099274,0.586110,1.000000,0.811953
price,0.046571,0.694556,0.811953,1.000000


In [111]:
x = df.drop('price', axis=1)
y = df['price']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8, random_state=0)
ss.fit(x_train)
x_train=ss.transform(x_train)
ss.fit(x_test)
x_test=ss.transform(x_test)

In [112]:
model=LinearRegression()
model.fit(x_train,y_train)
print('訓練データに対する決定係数：', model.score(x_train, y_train))
print('テストデータに対する決定係数：', model.score(x_test, y_test))

訓練データに対する決定係数： 0.6710000233941398
テストデータに対する決定係数： 0.5172369180997686


考察：訓練データの決定係数が0.67とやや低く、また、決定係数の差が大きいため、学習がやや離れているため、過学習の傾向が若干見られる。
説明変数と目的変数の相関係数がドアの数で大きく異なっていることが原因

# 5. 模範解答

```
# コード1（模範解答は最下部）
# csv ファイルを pandas の DataFrame オブジェクト df として読み込み
df = pd.read_csv('autos_data.csv')
```

```
# コード2（模範解答は最下部）
# df を表示
df
```

```
# コード3（模範解答は最下部）
# 4 つのカラム「horsepower」、「width」、「height」、「price」以外を削除
df = df[['price', 'horsepower', 'width', 'height']]

# df を表示（確認）
df
```

```
# コード4（模範解答は最下部）
# df のデータ型の確認
df.dtypes
```

```
# コード5（模範解答は最下部）
# 各カラム（列）同士の相関係数の計算
df.corr()
```

```
# コード6（模範解答は最下部）
# データを標準化するライブラリのインポート
from sklearn import preprocessing

# x_train を標準化し x_train に格納
ss = preprocessing.StandardScaler()
ss.fit(x_train)
x_train = ss.transform(x_train)

# x_test を標準化し x_test に格納
x_test= ss.transform(x_test)

# x_train を表示
x_train
```

```
# コード7（模範解答は最下部）
# モデルの初期化と学習
model = LinearRegression()
model.fit(x_train, y_train)
```